In [1]:
import sys
sys.path.insert(0, "../")

In [2]:
import numpy as np
import netket as nk
import jax

In [3]:
from spinlattice.lattice import get_lattice_data

In [4]:
lt_info = get_lattice_data("dimer-hexagonal-lattice", "dimer-hexagonal", [2, 2,], "periodic")

In [5]:
import os
os.environ["JAX_PLATFORM_NAME"] = "cpu"


In [7]:
N = lt_info.n_sites()
hi = nk.hilbert.Spin(s=1 / 2, N=N)

In [10]:
from netket.operator.spin import sigmax,sigmaz 


Gamma=[1, -1]
H = sum([Gamma[c]*sigmax(hi,e[0]) * sigmax(hi, e[1]) for e, c in zip(lt_info.bonds, lt_info.bond_types)])



In [16]:
H._setup()
H._mels

array([[[-1.],
        [-1.],
        [-1.],
        [-1.]],

       [[ 1.],
        [ 1.],
        [ 1.],
        [ 1.]],

       [[ 1.],
        [ 1.],
        [ 1.],
        [ 1.]],

       [[ 1.],
        [ 1.],
        [ 1.],
        [ 1.]],

       [[ 1.],
        [ 1.],
        [ 1.],
        [ 1.]],

       [[ 1.],
        [ 1.],
        [ 1.],
        [ 1.]],

       [[-1.],
        [-1.],
        [-1.],
        [-1.]],

       [[ 1.],
        [ 1.],
        [ 1.],
        [ 1.]],

       [[ 1.],
        [ 1.],
        [ 1.],
        [ 1.]],

       [[ 1.],
        [ 1.],
        [ 1.],
        [ 1.]],

       [[ 1.],
        [ 1.],
        [ 1.],
        [ 1.]],

       [[ 1.],
        [ 1.],
        [ 1.],
        [ 1.]],

       [[-1.],
        [-1.],
        [-1.],
        [-1.]],

       [[ 1.],
        [ 1.],
        [ 1.],
        [ 1.]],

       [[ 1.],
        [ 1.],
        [ 1.],
        [ 1.]],

       [[ 1.],
        [ 1.],
        [ 1.],
        [ 1.]],

       [

In [17]:
H._acting_on

array([[ 0,  1],
       [ 1,  2],
       [ 2,  3],
       [ 2,  9],
       [ 3,  4],
       [ 3, 12],
       [ 4,  5],
       [ 5,  6],
       [ 6,  7],
       [ 6, 13],
       [ 0,  7],
       [ 7,  8],
       [ 8,  9],
       [ 9, 10],
       [10, 11],
       [ 1, 10],
       [11, 12],
       [ 4, 11],
       [12, 13],
       [13, 14],
       [14, 15],
       [ 5, 14],
       [ 8, 15],
       [ 0, 15]])

In [19]:
lt_info.bonds

array([[ 0,  1],
       [ 1,  2],
       [ 2,  3],
       [ 2,  9],
       [ 3,  4],
       [ 3, 12],
       [ 4,  5],
       [ 5,  6],
       [ 6,  7],
       [ 6, 13],
       [ 7,  0],
       [ 7,  8],
       [ 8,  9],
       [ 9, 10],
       [10, 11],
       [10,  1],
       [11, 12],
       [11,  4],
       [12, 13],
       [13, 14],
       [14, 15],
       [14,  5],
       [15,  8],
       [15,  0]])